**Q1.** Write Ansible playbooks to automate the setup and configuration of a web server (e.g., Apache or Nginx)?

**Ans:** Step 1: Launch EC2 dynamically using Ansible

Ansible has the amazon.aws.ec2 module to create EC2 instances. You need:

AWS CLI configured (aws configure)

Ansible collections: amazon.aws installed (ansible-galaxy collection install amazon.aws)

In [ ]:
# Playbook: launch_ec2.yml

- name: Launch EC2 Instance
  hosts: localhost
  gather_facts: false
  tasks:
    - name: Launch EC2 instance
      amazon.aws.ec2:
        key_name: my-keypair
        instance_type: t2.micro
        image_id: ami-0abcdef1234567890   # Use correct AMI ID for your region
        wait: yes
        region: us-east-1
        count: 1
        instance_tags:
          Name: Ansible-WebServer
      register: ec2

    - name: Add new instance to host group
      add_host:
        hostname: "{{ item.public_ip }}"
        groupname: launched
      loop: "{{ ec2.instances }}"

    - name: Wait for SSH to come up
      wait_for:
        host: "{{ item.public_ip }}"
        port: 22
        delay: 10
        timeout: 300
      loop: "{{ ec2.instances }}"

**Step 2:** Configure Web Server on EC2 Now you can create another playbook that runs on the newly launched instance.

In [ ]:
# Playbook: webserver_setup.yml

- name: Setup Web Server
  hosts: launched
  become: yes
  gather_facts: yes

  tasks:
    - name: Update apt cache
      apt:
        update_cache: yes

    - name: Install Apache
      apt:
        name: apache2
        state: present

    - name: Ensure Apache service is running
      service:
        name: apache2
        state: started
        enabled: yes

    - name: Deploy custom index.html
      copy:
        content: "Hello from Ansible Managed EC2!"
        dest: /var/www/html/index.html

**Step 3:** Run Both Playbooks Together You can create a main.yml to run sequentially:

In [ ]:

# Playbook: main.yml
---
- import_playbook: launch_ec2.yml
- import_playbook: webserver_setup.yml

**Step 4:** Now Run this command on your CLI to run all ansible-playbook is:

In [ ]:
ansible-playbook main.yml